# CRNN_OCR_Paper

- 참고 사이트
    - [[한글 OCR] CRNN Model 학습](https://velog.io/@shasha/%ED%95%9C%EA%B8%80-OCR-CRNN-Model-%ED%95%99%EC%8A%B5)
    - [딥러닝을 활용한 한글문장 OCR 프로젝트](https://medium.com/@sunwoopark/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5-ocr-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-hclt-2019-bb9d17622412)
    - [[python] CRNN 한글 모델 학습하기(1) - AIhub 데이터셋](https://velog.io/@apphia39/python-CRNN-%ED%95%9C%EA%B8%80-%EB%AA%A8%EB%8D%B8-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0)
    - [[#04] AI Hub 한국어 글자체 AI 이미지 데이터 전처리](https://cvml.tistory.com/21)
    - [LMDB 형식 데이터셋이란? | 텍스트 인식 데이터셋 포맷](https://mvje.tistory.com/139)

## CRNN 구조
- Convolution Layers : 입력 이미지로부터 feature 시퀀스 추출 ( -> CNNs )
- Recurrent Layers : 각 프레임마다 라벨 예측 ( -> RNNs )
- Transcriptional Layers : 프레임마다의 예측을 최종 라벨 시퀀스로 변경

# 실제 코드

## file 구조
- 원본 파일 순서대로 사용할 것
- json 파일 전처리 부분 재구성할 것

In [1]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import glob

import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [2]:
# JSON 파일 병합 함수
def merge_json_files(input_dir, output_file):
    merged_data = []
    json_pattern = os.path.join(input_dir, '*.json')
    file_list = glob.glob(json_pattern)
    
    for file in file_list:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.append(data)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, indent=4)

# 입력 디렉토리와 출력 파일 경로 설정

TL_path = 'data/OCR_Bulk/TL/P.Paper/'
VL_path = 'data/OCR_Bulk/VL/P.Paper/'

folders = ['O.Form/', 'R.Free/']

def FileList(path, folders):
    
    for folder in folders:
        input_directory = path + folder

        output_file = 'merged_labels_paper_Train_' + folder[:1] + '.json'

        # JSON 파일 병합 실행
        merge_json_files(input_directory, output_file)
        print(f"Merged JSON data written to '{output_file}'")


In [ ]:
# FileList(VL_path, folders)

In [ ]:
# FileList(TL_path, folders)

In [3]:
# JSON 파일 로드 함수
def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 이미지 전처리 함수
def preprocess_image(image_path, bounding_box):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     print(image)
    x, y, w, h = bounding_box
#     print(x, y, w, h)
    cropped_image = image[y:y+h, x:x+w]
#     print('crop')
    resized_image = cv2.resize(cropped_image, (128, 32))  # CRNN 입력 크기에 맞게 조정
    normalized_image = resized_image / 255.0
    return normalized_image

# JSON 파일 경로
# json_path = merged_labels_paper_Train_R.json
json_path = 'merged_labels_paper_Train_O.json'
data = load_json(json_path)

# 각 단어의 바운딩 박스와 텍스트 정보
words_info = data[0]['bbox']

# 이미지 경로
image_path = 'data/OCR_Bulk/VS/P.Paper/O.Form/' + data[0]['Images']['identifier'] + '.' + data[0]['Images']['type']

# 전처리된 이미지 리스트
processed_images = []
for word_info in words_info:

    # x, y, w, h
    bounding_box = [word_info['x'][0], 
                    word_info['y'][0],
                    word_info['x'][2] - word_info['x'][0], 
                    word_info['y'][1] - word_info['y'][0]
                   ]
    processed_image = preprocess_image(image_path, bounding_box)
    processed_images.append(processed_image)


In [ ]:
# def preprocess_image(image_path, bounding_box):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
# #     cv2.imshow('rgb_image', image)
# #     print(image)
#     x, y, w, h = bounding_box
# #     print(x, y, w, h)
#     cropped_image = image[y:y+h, x:x+w]
# #     print('crop')
#     resized_image = cv2.resize(cropped_image, (128, 32))  # CRNN 입력 크기에 맞게 조정
#     normalized_image = resized_image / 255.0
#     return normalized_image

# for word_info in words_info:

#     # x, y, w, h
#     bounding_box = [word_info['x'][0], 
#                     word_info['y'][0],
#                     word_info['x'][2] - word_info['x'][0], 
#                     word_info['y'][1] - word_info['y'][0]
#                    ]
#     processed_image = preprocess_image(image_path, bounding_box)
#     processed_images.append(processed_image)


In [ ]:
# # 각 단어의 바운딩 박스와 텍스트 정보
# words_info = data[0]['bbox']

# # 이미지 경로
# image_path = data[0]['Images']['identifier'] + '.' + data[0]['Images']['type']

# # 전처리된 이미지 리스트
# processed_images = []
# for word_info in words_info:

#     bounding_box = [word_info['x'][0], 
#                     word_info['y'][0],
#                     word_info['x'][2] - word_info['x'][0], 
#                     word_info['y'][1] - word_info['y'][0]
#                    ]
#     print(bounding_box)

In [4]:
# CTC 디코딩 함수
def ctc_decode(predictions, charset):
    decoded = K.ctc_decode(predictions, input_length=np.ones(predictions.shape[0]) * predictions.shape[1])[0][0]
    decoded_text = K.get_value(decoded)
    return ''.join([charset[i] for i in decoded_text if i != -1])

# CRNN 모델 정의 (간단한 예제)
# Conv-Pool layer 갯수 = 찾을 특성의 갯수
# padding: 사이즈 유지를 위해 conv전에 0을 모서리에 추가함
# drop out: 일부러 정보를 누락, 중간 노드를 꺼서 과적합을 방지하는데에 사용
    # 응용력을 주는 것
def build_crnn_model(input_shape, num_classes):
    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Reshape((-1, x.shape[-1]))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    return model

# 모델 입력 크기와 클래스 수 정의
input_shape = (32, 128, 1)
num_classes = 2350  # 예: 한글 음절 수

# 모델 빌드
model = build_crnn_model(input_shape, num_classes)

# 모델 요약 출력
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 128, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 128, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 32, 64)         0         
 g2D)                                                            
                                                                 
 reshape (Reshape)           (None, 256, 64)           0     

charset 값으로 들어가야 하는 실제 한글 음절 셋은 CRNN 모델이 예측한 인덱스를 실제 한글 문자로 변환하기 위한 문자 집합입니다. 한글 음절 셋은 모델이 인식할 수 있는 모든 한글 음절을 포함해야 합니다. 예를 들어, 한글 음절은 ‘가’, ‘나’, ‘다’, ‘라’ 등으로 구성됩니다.

- Input Layer:
    - input_1 (InputLayer): 입력 데이터의 형태는 (None, 32, 128, 1)입니다. 여기서 None은 배치 크기를 의미하며, (32, 128, 1)은 이미지의 높이, 너비, 채널 수를 나타냅니다.
- Conv2D Layer:
    - conv2d (Conv2D): 첫 번째 합성곱 층으로, 32개의 필터를 사용하여 (3, 3) 크기의 커널을 적용합니다. 출력 형태는 (None, 32, 128, 32)이며, 파라미터 수는 320입니다.
    - max_pooling2d (MaxPooling2D): 첫 번째 풀링 층으로, (2, 2) 크기의 풀링을 적용합니다. 출력 형태는 (None, 16, 64, 32)입니다.
- Conv2D Layer:
    - conv2d_1 (Conv2D): 두 번째 합성곱 층으로, 64개의 필터를 사용하여 (3, 3) 크기의 커널을 적용합니다. 출력 형태는 (None, 16, 64, 64)이며, 파라미터 수는 18,496입니다.
    - max_pooling2d_1 (MaxPooling2D): 두 번째 풀링 층으로, (2, 2) 크기의 풀링을 적용합니다. 출력 형태는 (None, 8, 32, 64)입니다.
- Reshape Layer:
    - reshape (Reshape): 데이터를 (None, 256, 64) 형태로 재구성합니다. 여기서 256은 타임스텝 수, 64는 특징 맵의 수입니다.
- Bidirectional LSTM Layer:
    - bidirectional (Bidirectional): 양방향 LSTM 층으로, 각 방향에 128개의 유닛을 사용합니다. 출력 형태는 (None, 256, 256)이며, 파라미터 수는 197,632입니다.
- Dense Layer:
    - dense (Dense): 출력 층으로, 2350개의 클래스(한글 음절 수)를 예측합니다. 출력 형태는 (None, 256, 2350)이며, 파라미터 수는 603,950입니다.
- 총 파라미터 수
    - Total params: 820,398
    - Trainable params: 820,398
    - Non-trainable params: 0
- 이 요약 결과는 모델의 각 층이 어떻게 구성되어 있는지, 각 층의 출력 형태와 파라미터 수를 보여줍니다. 이를 통해 모델의 복잡도와 메모리 요구 사항을 파악할 수 있습니다.

In [5]:
# 예측 수행 및 CTC 디코딩
# charset = '가나다라마바사아자차카타파하'  # 예시 문자셋 (실제 한글 음절 셋으로 대체 필요)

# 한글 음절 셋 생성 코드
def generate_hangul_syllables():
    hangul_syllables = []
    for cho in range(0x1100, 0x1113):  # 초성 (19자)
        for jung in range(0x1161, 0x1176):  # 중성 (21자)
            for jong in range(0x11A8, 0x11C3):  # 종성 (27자)
                syllable = chr(0xAC00 + (cho - 0x1100) * 21 * 28 + (jung - 0x1161) * 28 + (jong - 0x11A7))
                hangul_syllables.append(syllable)
    return hangul_syllables

# 한글 음절 셋 생성
hangul_syllables = generate_hangul_syllables()

# 생성된 한글 음절 셋 출력 (일부 예시)
# print(hangul_syllables[:100])  # 처음 100개의 음절 출력

# 한문, 영어, 특수문자는 일단 제외함

charset = hangul_syllables

for processed_image in processed_images:
    print('예측 수행')
    processed_image = np.expand_dims(processed_image, axis=-1)  # 채널 차원 추가
    processed_image = np.expand_dims(processed_image, axis=0)  # 배치 차원 추가
    print('예측값 선언')
    predictions = model.predict(processed_image)
    
    # CTC 디코딩을 통해 실제 텍스트 판독
    print('디코딩')
    predicted_text = ctc_decode(predictions, charset)
    print(f"Predicted text: {predicted_text}")

예측 수행
예측값 선언
1/1 [==============================] - 1s 1s/step
디코딩


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
predictions

array([[[0.00043285, 0.00042642, 0.00042254, ..., 0.00041453,
         0.00043274, 0.00042946],
        [0.00043297, 0.00042556, 0.00042413, ..., 0.00041413,
         0.00043265, 0.00042948],
        [0.00043272, 0.00042511, 0.00042492, ..., 0.00041373,
         0.00043256, 0.00042947],
        ...,
        [0.00042761, 0.00042256, 0.00043273, ..., 0.00041754,
         0.00042574, 0.00042679],
        [0.00042666, 0.00042218, 0.000433  , ..., 0.0004184 ,
         0.0004248 , 0.00042649],
        [0.00042467, 0.00042201, 0.0004336 , ..., 0.00041994,
         0.00042275, 0.00042562]]], dtype=float32)

In [8]:
processed_image

array([[[[1.        ],
         [1.        ],
         [1.        ],
         ...,
         [0.99215686],
         [0.99215686],
         [0.99215686]],

        [[1.        ],
         [1.        ],
         [1.        ],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        [[0.99607843],
         [1.        ],
         [1.        ],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        ...,

        [[1.        ],
         [1.        ],
         [1.        ],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        [[0.99607843],
         [0.99607843],
         [0.99607843],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        [[0.99215686],
         [0.99215686],
         [0.99215686],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]]]])